In [1]:
import pandas as pd
import pickle5 as pickle
import numpy as np
import matplotlib.pyplot as plt

# Load
SDIR = "../data/Red-lines5-formodeling-210309_102050"
SDIR = "/data2/analyses/database/expts/Red-lines5-formodeling-210309_102050"

with open(f"{SDIR}/dat.pkl", "rb") as f:
    dat = pickle.load(f)

In [22]:
import torch
from pybpl.data import unif_space
from pybpl.splines import fit_bspline_to_traj

def fit_minimal_spline(stroke, thresh=0.7, max_nland=100, normalize=True):
    stroke = torch.from_numpy(stroke).float()
    ntraj = stroke.size(0)
    max_nland = min(max_nland, ntraj+1)
    
    # determine num control points
    for nland in range(1, max_nland):
        spline, residuals = fit_bspline_to_traj(stroke, nland, include_resid=True)
        loss = residuals.sum()
        if normalize:
            loss = loss / ntraj
        if loss < thresh:
            return spline
    

    return spline


def fit_spline(stroke, nland=7):
    stroke = torch.from_numpy(stroke).float()
    spline = fit_bspline_to_traj(stroke, nland)
    return spline


In [3]:
print(dat["strokes_task"].shape)

(5599,)


In [23]:
splines_list = []
# for i in range(dat["strokes_task"].shape[0]):
for i in range(50): # load fraction of data since loading everything takes a while
    if i % 100 == 0:
        print(i)
    strokes = dat["strokes_task"].values[i]
    splines = []
    for stroke in strokes:
        stroke = stroke[:, :2] # get rid of time dimension
        stroke = unif_space(stroke) # make sure stroke uniformly sampled in space
        if False:
            spline = fit_minimal_spline(stroke)
        else:
            spline = fit_spline(stroke)
        
        splines.append(spline)
    splines_list.append(splines)

0


In [24]:
print(len(splines_list))
for spline in splines_list[0]:
    print(spline.shape)

50
torch.Size([7, 2])
torch.Size([7, 2])
torch.Size([7, 2])


In [25]:
from gns.type.model import TypeModel
model = TypeModel()
print(model._losses_fn(splines_list))

tensor([892.8423, 696.2638, 209.6604, 723.0002, 656.4944, 385.9243, 601.3422,
        440.5765, 449.8077, 888.3727, 245.8254, 174.3480, 189.1565, 465.3210,
        321.7867, 549.8481, 225.5410, 417.9521, 738.8586, 709.0200, 588.2217,
        212.2838, 277.5056, 224.2545, 285.9850, 175.4550, 387.2458, 521.2498,
        720.2775, 400.8983, 504.6115, 810.2320, 610.5807, 998.8503, 467.0151,
        237.1090, 697.1028, 410.8200, 620.4103, 733.8641, 296.7352, 674.4497,
        505.1075, 826.8172, 690.1113, 227.9642, 830.4385, 649.6740, 550.5046,
        485.9715])


In [26]:
num_images = 50
train_sz = int(num_images * 0.8)
val_sz = num_images - train_sz
train_data = splines_list[0:train_sz]
val_data = splines_list[train_sz:train_sz+val_sz]

In [27]:
import torch
import torch.utils.data as data
import torch.optim as optim


In [28]:
train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=5,
    shuffle=True,
    pin_memory=torch.cuda.is_available()
)

In [29]:
val_loader = data.DataLoader(
    dataset=val_data,
    batch_size=5,
    shuffle=False,
    pin_memory=torch.cuda.is_available()
)

In [30]:
weight_decay = 0.001
optimizer = optim.Adam(
    model.parameters(),
    weight_decay=weight_decay
)

epochs = 10
device = 'cpu'
saver = None # todo: change

In [31]:
for a in train_loader:
    print(a)

[tensor([[[  88.3199,   51.2000],
         [ 113.9141,   51.2000],
         [ 132.6474,   51.2000],
         [ 153.6001,   51.2000],
         [ 174.5526,   51.2000],
         [ 193.2858,   51.2000],
         [ 218.8801,   51.2000]],

        [[-153.5998,  103.6098],
         [-153.5997,  153.1646],
         [-153.5999,  189.6868],
         [-153.6000,  230.4000],
         [-153.6003,  271.1136],
         [-153.6002,  307.6353],
         [-153.6005,  357.1905]],

        [[ -75.9123, -127.9992],
         [-106.3404, -127.9996],
         [-128.6615, -127.9998],
         [-153.6001, -128.0000],
         [-178.5385, -128.0002],
         [-200.8596, -128.0004],
         [-231.2877, -128.0007]],

        [[-153.6010,  -22.9217],
         [-153.6007,  -64.0192],
         [-153.6003,  -94.2619],
         [-153.6001, -128.0001],
         [-153.5997, -161.7381],
         [-153.5993, -191.9809],
         [-153.5989, -233.0782]],

        [[-256.0960,  230.4001],
         [-216.0042,  230.3999],
 

In [12]:
from ptkit.training import train_model
metrics = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    epochs=epochs,
    device=device,
    saver=saver,
    loss_fn=model.loss_fn,
)

  0%|          | 0/3 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [2, 2] at entry 0 and [6, 2] at entry 1